# Imports and Preprocessing

In [ ]:
# Install BIOGEME using pip
#pip install biogeme

In [2]:
#check BIOGEME version
import biogeme.version as ver
print("Biogeme version:", ver.getVersion())

#Import necessary libraries
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable

Biogeme version: 3.2.11


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Important!: All data have to be numeric**

In [3]:
# Load the database
df = pd.read_csv("ModeChoiceData_RP.csv", sep=',') # Check the path to your CSV file

#create backup of the original dataframe
df_backup = df.copy()

In [4]:
# Check missing values
print(df.isnull().sum())



Unnamed: 0        0
ID                0
RP                0
SP                0
RP_journey        0
SP_task         512
av_car            0
av_rail           0
time_car          0
cost_car          0
time_rail         0
cost_rail         0
business          0
access_rail       0
service_rail      0
female            0
income            0
choice            0
dtype: int64


In [5]:
# Drop columns with missing values
## df = df.drop(columns=["SP_task"])
# Or, set the missing values to -1
df = df.fillna(-1)
print(df.isnull().sum())

# Additional preprocessing the data
# Create a row index
df["RowID"] = df.index + 1
#df

Unnamed: 0      0
ID              0
RP              0
SP              0
RP_journey      0
SP_task         0
av_car          0
av_rail         0
time_car        0
cost_car        0
time_rail       0
cost_rail       0
business        0
access_rail     0
service_rail    0
female          0
income          0
choice          0
dtype: int64


In [6]:
# Define the BIOGEME-specific database
database = db.Database("ModeChoice", df)
#print the BIOGEME-specific database
database.data

,Unnamed: 0,ID,RP,SP,RP_journey,SP_task,av_car,av_rail,time_car,cost_car,time_rail,cost_rail,business,access_rail,service_rail,female,income,choice,RowID
0,129,9,1,0,1,-1.0,1,1,250,45,155,65,1,15,0,0,52.314,1,1
1,145,10,1,0,1,-1.0,1,1,345,45,130,35,0,5,0,0,66.143,1,2
2,146,10,1,0,2,-1.0,1,1,275,50,120,45,0,15,0,0,66.143,1,3
3,161,11,1,0,1,-1.0,1,1,300,30,155,75,0,25,0,1,26.718,1,4
4,178,12,1,0,2,-1.0,1,1,300,35,140,45,0,20,0,0,47.709,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,7938,497,1,0,2,-1.0,1,1,275,30,155,65,0,25,0,0,65.335,1,508
508,7954,498,1,0,2,-1.0,1,1,345,35,170,35,0,15,0,1,37.162,2,509
509,7969,499,1,0,1,-1.0,1,1,390,30,130,75,1,15,0,1,21.154,1,510
510,7985,500,1,0,1,-1.0,1,1,390,35,155,55,1,15,0,0,19.910,1,511


In [7]:
"""
Use the loop below to get all of columns names.
for col in df.columns:
    print(f'{col} = Variable("{col}")')

1. The code will generate the Variable for each column.    
2. Copy the output and paste it below to define the variables.
"""
for col in df.columns:
    print(f'{col} = Variable("{col}")')


Unnamed: 0 = Variable("Unnamed: 0")
ID = Variable("ID")
RP = Variable("RP")
SP = Variable("SP")
RP_journey = Variable("RP_journey")
SP_task = Variable("SP_task")
av_car = Variable("av_car")
av_rail = Variable("av_rail")
time_car = Variable("time_car")
cost_car = Variable("cost_car")
time_rail = Variable("time_rail")
cost_rail = Variable("cost_rail")
business = Variable("business")
access_rail = Variable("access_rail")
service_rail = Variable("service_rail")
female = Variable("female")
income = Variable("income")
choice = Variable("choice")
RowID = Variable("RowID")


In [8]:
#Define the variables
ID = Variable("ID")
RP = Variable("RP")
SP = Variable("SP")
RP_journey = Variable("RP_journey")
av_car = Variable("av_car")
av_rail = Variable("av_rail")
time_car = Variable("time_car")
cost_car = Variable("cost_car")
time_rail = Variable("time_rail")
cost_rail = Variable("cost_rail")
business = Variable("business")
access_rail = Variable("access_rail")
service_rail = Variable("service_rail")
female = Variable("female")
income = Variable("income")
choice = Variable("choice")

# Binary Logit Models

#### Specification Baseline (Model A from Lecture 3)

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)

# Utility functions
V_car = asc_car + b_cost * cost_car
V_rail = asc_rail + b_cost * cost_rail

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_Base' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

Results for model Model_A
Nbr of parameters:		2
Sample size:			512
Excluded data:			0
Null log likelihood:		-354.8914
Final log likelihood:		-344.5628
Likelihood ratio test (null):		20.65708
Rho square (null):			0.0291
Rho bar square (null):			0.0235
Akaike Information Criterion:	693.1256
Bayesian Information Criterion:	701.6023



,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_rail,0.126523,0.119387,1.059767,0.289250
b_cost,-0.022415,0.005687,-3.941437,0.000081


#### Specification 1 (Model B from Lecture 3)

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)
b_tt = Beta('b_tt', 0, None, None, 0)

# Utility functions
V_car = asc_car + b_cost * cost_car + b_tt * time_car
V_rail = asc_rail + b_cost * cost_rail + b_tt * time_rail

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_1' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

#### Specification 2 (Alternative-Specific)

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)
b_tt_car = Beta('b_tt_car', 0, None, None, 0)
b_tt_rail = Beta('b_tt_rail', 0, None, None, 0)

# Utility functions
V_car = asc_car + b_cost * cost_car + b_tt_car * time_car
V_rail = asc_rail + b_cost * cost_rail + b_tt_rail * time_rail

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_2' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

#### Specification 3

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)
b_tt_car = Beta('b_tt_car', 0, None, None, 0)
b_tt_rail = Beta('b_tt_rail', 0, None, None, 0)
b_female_rail = Beta('b_female_rail', 0, None, None, 0)

# Utility functions
V_car = asc_car + b_cost * cost_car + b_tt_car * time_car
V_rail = asc_rail + b_cost * cost_rail + b_tt_rail * time_rail + b_female_rail * female

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_3' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

#### Specification 4

Since income level doesn't exist originally, we need to update the dataset

In [9]:
# Update the database with income levels
df['income_level'] = pd.cut(df['income'], bins=[0, 35, 55, df['income'].max()], labels=[1, 2, 3])
# Change income_level type from categorical to integer
df['income_level'] = df['income_level'].astype(int)
df



,Unnamed: 0,ID,RP,SP,RP_journey,SP_task,av_car,av_rail,time_car,cost_car,time_rail,cost_rail,business,access_rail,service_rail,female,income,choice,RowID,income_level
0,129,9,1,0,1,-1.0,1,1,250,45,155,65,1,15,0,0,52.314,1,1,2
1,145,10,1,0,1,-1.0,1,1,345,45,130,35,0,5,0,0,66.143,1,2,3
2,146,10,1,0,2,-1.0,1,1,275,50,120,45,0,15,0,0,66.143,1,3,3
3,161,11,1,0,1,-1.0,1,1,300,30,155,75,0,25,0,1,26.718,1,4,1
4,178,12,1,0,2,-1.0,1,1,300,35,140,45,0,20,0,0,47.709,1,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,7938,497,1,0,2,-1.0,1,1,275,30,155,65,0,25,0,0,65.335,1,508,3
508,7954,498,1,0,2,-1.0,1,1,345,35,170,35,0,15,0,1,37.162,2,509,2
509,7969,499,1,0,1,-1.0,1,1,390,30,130,75,1,15,0,1,21.154,1,510,1
510,7985,500,1,0,1,-1.0,1,1,390,35,155,55,1,15,0,0,19.910,1,511,1


In [10]:
# Update the BIOGEME-specific database
database = db.Database("ModeChoice", df)
database.data

,Unnamed: 0,ID,RP,SP,RP_journey,SP_task,av_car,av_rail,time_car,cost_car,time_rail,cost_rail,business,access_rail,service_rail,female,income,choice,RowID,income_level
0,129,9,1,0,1,-1.0,1,1,250,45,155,65,1,15,0,0,52.314,1,1,2
1,145,10,1,0,1,-1.0,1,1,345,45,130,35,0,5,0,0,66.143,1,2,3
2,146,10,1,0,2,-1.0,1,1,275,50,120,45,0,15,0,0,66.143,1,3,3
3,161,11,1,0,1,-1.0,1,1,300,30,155,75,0,25,0,1,26.718,1,4,1
4,178,12,1,0,2,-1.0,1,1,300,35,140,45,0,20,0,0,47.709,1,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,7938,497,1,0,2,-1.0,1,1,275,30,155,65,0,25,0,0,65.335,1,508,3
508,7954,498,1,0,2,-1.0,1,1,345,35,170,35,0,15,0,1,37.162,2,509,2
509,7969,499,1,0,1,-1.0,1,1,390,30,130,75,1,15,0,1,21.154,1,510,1
510,7985,500,1,0,1,-1.0,1,1,390,35,155,55,1,15,0,0,19.910,1,511,1


In [ ]:
#Update Variable definitions for income levels
#Hint: if you have multiple new columns,
# you can re run the loop function to get the variable definitions, 
# then copy the output and paste it below to define the new variables.
income_level = Variable("income_level")

Unnamed: 0 = Variable("Unnamed: 0")
ID = Variable("ID")
RP = Variable("RP")
SP = Variable("SP")
RP_journey = Variable("RP_journey")
SP_task = Variable("SP_task")
av_car = Variable("av_car")
av_rail = Variable("av_rail")
time_car = Variable("time_car")
cost_car = Variable("cost_car")
time_rail = Variable("time_rail")
cost_rail = Variable("cost_rail")
business = Variable("business")
access_rail = Variable("access_rail")
service_rail = Variable("service_rail")
female = Variable("female")
income = Variable("income")
choice = Variable("choice")
RowID = Variable("RowID")
income_level = Variable("income_level")


In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)

b_MI_rail = Beta('b_mid_rail', 0, None, None, 0) # New parameter for mid income
b_HI_rail = Beta('b_high_rail', 0, None, None, 0) # New parameter for high income

# Utility functions
V_car = asc_car + b_cost * cost_car
V_rail = asc_rail + b_cost * cost_rail  + b_MI_rail * (income_level == 2) + b_HI_rail * (income_level == 3)

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_4' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

Results for model Alternative-specific
Nbr of parameters:		4
Sample size:			512
Excluded data:			0
Null log likelihood:		-354.8914
Final log likelihood:		-344.42
Likelihood ratio test (null):		20.94279
Rho square (null):			0.0295
Rho bar square (null):			0.0182
Akaike Information Criterion:	696.8399
Bayesian Information Criterion:	713.7932



,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_rail,0.167528,0.170709,0.981362,0.326414
b_cost,-0.022503,0.005692,-3.953420,0.000077
b_high_rail,-0.015860,0.216838,-0.073143,0.941693
b_mid_rail,-0.111004,0.222413,-0.499087,0.617718


#### Specification 5

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)
b_tt_car = Beta('b_tt_car', 0, None, None, 0)
b_tt_rail = Beta('b_tt_rail', 0, None, None, 0)
b_female_rail = Beta('b_female_rail', 0, None, None, 0)
b_business_rail = Beta('b_business_rail', 0, None, None, 0)
busi_inc_rail = Beta('busi_inc_rail', 0, None, None, 0)

# Utility functions
V_car = (asc_car + 
         b_cost * cost_car + 
         b_tt_car * time_car
        )
V_rail = (asc_rail + 
          b_cost * cost_rail + 
          b_tt_rail * time_rail + 
          b_female_rail * female +
          b_business_rail * business +
          busi_inc_rail * income * business
          )

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_5' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

#### Specification 6

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)
income_rail = Beta('income_rail', 0, None, None, 0) # New parameter for income effect on rail
b_cost_inc = Beta('b_cost_inc', 0, None, None, 0) # New parameter for cost effect on income

# Utility functions
V_car = (asc_car + 
         b_cost * cost_car + 
         b_cost_inc * cost_car / income
        )
V_rail = (asc_rail + 
          b_cost * cost_rail + 
          income_rail * income +
          b_cost_inc * cost_rail / income
        )

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_6' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

Results for model Alternative-specific
Nbr of parameters:		4
Sample size:			512
Excluded data:			0
Null log likelihood:		-354.8914
Final log likelihood:		-344.1666
Likelihood ratio test (null):		21.44953
Rho square (null):			0.0302
Rho bar square (null):			0.0189
Akaike Information Criterion:	696.3332
Bayesian Information Criterion:	713.2865



,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_rail,0.336132,0.318177,1.056431,0.290772
b_cost,-0.013056,0.012216,-1.068834,0.285144
b_cost_inc,-0.341591,0.394282,-0.866361,0.386292
income_rail,-0.004752,0.006633,-0.716422,0.473731


#### Log-Transformation

In [37]:
import numpy as np
# Update the dataset with log-transformed cost variables
df['log_cost_car'] = np.log(df['cost_car'])
df['log_cost_rail'] = np.log(df['cost_rail'])
# Update the BIOGEME-specific database
database = db.Database("ModeChoice", df)
database.data
#update Variables
log_cost_car = Variable("log_cost_car")
log_cost_rail = Variable("log_cost_rail")

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)

# Utility functions
V_car = (asc_car + 
         b_cost * log_cost_car
        )
V_rail = (asc_rail + 
          b_cost * log_cost_rail
        )

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_Log' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

Results for model Alternative-specific
Nbr of parameters:		2
Sample size:			512
Excluded data:			0
Null log likelihood:		-354.8914
Final log likelihood:		-344.9117
Likelihood ratio test (null):		19.95929
Rho square (null):			0.0281
Rho bar square (null):			0.0225
Akaike Information Criterion:	693.8234
Bayesian Information Criterion:	702.3001



,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_rail,0.105359,0.117198,0.898985,0.368661
b_cost,-1.051321,0.273168,-3.848629,0.000119


#### Power Expansion Series

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_rail = Beta('asc_rail', 0, None, None, 0) 
b_cost = Beta('b_cost', 0, None, None, 0)
b_cost_squared = Beta('b_cost_squared', 0, None, None, 0)
b_cost_cubed = Beta('b_cost_cubed', 0, None, None, 0)

# Utility functions
V_car = (asc_car + 
         b_cost * cost_car +
         b_cost_squared * cost_car * cost_car +
         b_cost_cubed * cost_car * cost_car * cost_car
        )
V_rail = (asc_rail + 
          b_cost * cost_rail +
          b_cost_squared * cost_rail * cost_rail +
          b_cost_cubed * cost_rail * cost_rail * cost_rail
        )

'''
Remember the coding of the alternatives in the dataset:
1: Car
2: Rail
''' 

# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'Spec_Pwr' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculateNullLoglikelihood(av)

# Save the estimation results
results = the_biogeme.estimate()

#Print the results
print(results.shortSummary())
results.getEstimatedParameters()

Results for model Alternative-specific
Nbr of parameters:		4
Sample size:			512
Excluded data:			0
Null log likelihood:		-354.8914
Final log likelihood:		-344.5117
Likelihood ratio test (null):		20.75928
Rho square (null):			0.0292
Rho bar square (null):			0.018
Akaike Information Criterion:	697.0234
Bayesian Information Criterion:	713.9767



,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_rail,0.126664,0.123643,1.024433,0.305631
b_cost,-0.049173,0.274441,-0.179175,0.857800
b_cost_cubed,-0.000005,0.000035,-0.137670,0.890501
b_cost_squared,0.000649,0.005485,0.118337,0.905800


#### Piecewise Linearization

In [13]:
#Update the database with piecewise cost variables
df['cost_car_TC1'] = df['cost_car'].copy()
df['cost_car_TC1'] = df['cost_car_TC1'].apply(lambda x: 40 if df['cost_car_TC1'] >= 40 else x)
df
#df['cost_car_TC2'] = df['cost_car'].apply(lambda x: 1 if x < 5 else (2 if x < 10 else (3 if x < 15 else 4)))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

#### Improved Model